# Utilisation de jit

<img src="https://github.com/lsteffenel/cours_numba_2017/blob/master/figures/numba_blue_icon_rgb.png?raw=1" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*8 novembre 2017*</center>

Comme dit en introduction, Numba propose plusieurs fonctionnalités. La plus générale est la fonction jit. Nous allons dans la suite aborder l'ensemble des possibilités de celle-ci.

Numba est efficace sur des données numériques de type scalaire ou des tableaux NumPy. Contrairement à vos habitudes pour une utilisation efficace de NumPy (écriture vectorielle), il est indispensable de dérouler les boucles comme vous le feriez dans un langage bas niveau pour avoir de bonnes performances. 

Prenons l'exemple d'un calcul sommant les coefficients d'un vecteur. En NumPy, ça s'écrit

In [ ]:
import numpy as np

x = np.random.rand(1000000)
print(x.sum())

Le temps de calcul est

In [ ]:
%timeit x.sum()

Voyons ce que ça donne avec Numba.

In [ ]:
from numba import jit

In [ ]:
@jit
def numba_sum(x):
    res= 0
    for i in range(x.size):
        res += x[i]
    return res

In [ ]:
%timeit numba_sum(x)

Le message indiqué en première ligne est dû au fait que Numba compile la fonction lors de son premier appel. Du coup, l'exécution de la fonction *numba_sum* est beaucoup plus lente que les autres.

Les performances sont bien plus mauvaises avec Numba. Ceci est dû au fait que NumPy utilise déjà des fonctions optimisées pour faire la somme.

## Fonctions utiles

Le décorateur jit de Numba ajoute un certain nombre de fonctions offrant des informations pouvant s'avérer très utiles.

In [ ]:
@jit
def somme(a, b):
    return a + b

#### inspect_types

Cette méthode permet de savoir quelles sont les fonctions générées pour un certain type de données d'entrée tout en nous montrant l'inférence de type proposée par Numba.

In [ ]:
somme.inspect_types()

Nous voyons ici qu'il n'y a pas pour le moment de fonction compilée par Numba. Si nous l'appelons une première fois avec un certain type de données

In [ ]:
somme(1, 2)
somme.inspect_types()

In [ ]:
somme(1., 2.)
somme.inspect_types()

#### inspect_llvm

Vous pouvez également avoir accès à la représentation intermédiaire de LLVM.

In [ ]:
for k, v in somme.inspect_llvm().items():
    print(v)

#### inspect_asm

Vous pouvez également avoir accès à l'assembleur qui intervient dans la phase finale du processus.

In [ ]:
for k, v in somme.inspect_asm().items():
    print(v)

## Rappeler sa fonction Python initiale

Il est toujours possibe d'appeler la fonction Python initiale sans la couche Numba.

In [ ]:
somme.py_func(1, 2)

## Spécialiser sa fonction pour des types données

Si vous utilisez juste **@jit**, Numba créera pour vous une nouvelle fonction dès lors qu'il n'a pas à disposition la fonction compilée avec ces types. Vous pouvez néanmoins avoir envie que votre fonction ne marche que pour certains types et vous renvoie une erreur si les types ne sont pas supportés.

Prenons par exemple le produit, nous voulons que celui-ci ne fonctionne que sur des entiers scalaires ou des tableaux 1d d'entiers. 

In [ ]:
from numba import jit

@jit(['int32[:](int32[:], int32[:])',
      'int32(int32, int32)'])
def produit(a, b):
    return a*b

In [ ]:
produit(2, 3)

In [ ]:
produit(3., 2.2)

In [ ]:
import numpy as np

a = np.arange(10, dtype=np.int32)
b = np.arange(10, dtype=np.int32)

produit(a, b)

In [ ]:
import numpy as np

a = np.random.random(10)
b = np.random.random(10)

produit(a, b)

Les types reconnus par Numba sont les suivants

- void,
- intp, uintp,
- intc, uintc,
- int8, uint8, int16, uint16, int32, uint32, int64, uint64,
- float32, float64,
- complex64, complex128.

Pour les tableaux, il suffit de préciser chacune des dimensions par le caractère deux points (**:**). Par exemple

- float32[:] est un tableau de type float du langage C à une dimension,
- float64[:, :] est un tableau de type double du langage C à deux dimensions.

## Options de compilation

Numba permet de mettre certaines directives suplémentaires dans les arguments du décorateur permettant de changer le comportement de la compilation. Pour le décorateur **@jit**, il en existe trois

- **nopython** 

cette option indique qu'il n'y a pas d'objets Python dans la fonction à optimiser et que tout peut donc être fait en bas niveau. Si c'est le cas, le résultat est optimal. Essayez toujours de mettre ce mode. La compilation échouera si la fonction comporte des objets Python.

- **nogil** 

cette option permet de relâcher le Global Interpreter Lock (GIL). Elle est utilisée lorsque l'on veut utiliser la fonction en parallèle en utilisant des threads.

- **cache** 

si vous ne voulez pas que Numba recompile votre fonction à chaque lancement de votre script, vous pouvez utiliser cette fonctionnalité qui met dans un fichier la version optimisée.

## inlining

Il est possible d'appeler des fonctions optimisées par **@jit** à l'intérieur de fonctions faisant également appel à **jit**. Si c'est possible, Numba les remplace alors directement par les lignes de codes associées.

Par exemple

In [ ]:
import math
from numba import njit

@njit
def square(x):
    return x ** 2

@njit
def hypot(x, y):
    return math.sqrt(square(x) + square(y))

In [ ]:
hypot(2., 3.)

In [ ]:
for k, v in hypot.inspect_asm().items():
    print(v)

## Exercice

Dans toute la suite de ce tutoriel, nous allons optimiser un ensemble de fonctions travaillant sur des splines cubiques. Il ne sera pas nécessaire de connaître les maths qu'il y a derrière et nous ne rentrerons donc pas dans les détails (si ça vous intéresse, vous pouvez suivre ce [lien](http://www.aip.de/groups/soe/local/numres/bookcpdf/c3-3.pdf)).

Le choix de cette exercice vient du site [inconvergent](http://inconvergent.net/generative). En prenant une figure géométrique de départ et en y associant une spline cubique passant par un ensemble de points discrétisant celle-ci, il est possible par petites perturbations successives d'avoir de très jolies images. Comme ces deux exemples.

![Sand Spline](https://github.com/lsteffenel/cours_numba_2017/blob/master/figures/sandspline.png?raw=1)

Le code source est dicponible [ici](https://github.com/gouarin/cours_numba_2017/blob/master/examples/sandspline/origin.py). L'explication du problème se trouve dans ce [notebook](./spline.ipynb).

Lorsque l'on utilise des splines cubiques, il est nécessaire de calculer la dérivée seconde passant par un ensemble de points. La fonction qui permet de les calculer est la suivante.

In [ ]:
def cubic_spline(x, y):
    n = x.shape[0]
    u = np.zeros_like(y)
    y2 = np.zeros_like(y)

    dif = np.diff(x)
    sig = dif[:-1]/(x[2:]-x[:-2])
    
    u[1:-1] = (y[2:]- y[1:-1])/dif[1:] - (y[1:-1]-y[:-2])/dif[:-1]

    for i in range(1, n-1):
        p = sig[i-1]*y2[i-1] + 2.
        y2[i] = (sig[i-1]-1)/p
        u[i] = (6*u[i]/(x[i+1]-x[i-1])-sig[i-1]*u[i-1])/p
    
    for i in range(n-2, -1, -1):
        y2[i] = y2[i]*y2[i+1]+u[i]

    return y2

In [ ]:
import numpy as np

x = np.random.random(100)
y = np.random.random(100)

In [ ]:
cubic_spline(x, y)
%timeit cubic_spline(x, y)

#### Exercice 1

Essayer d'optimiser cette fonction en mettant juste **@jit**.

#### Exercice 2

Essayer de mettre **@jit(nopython=True)** ou **@njit**. Qu'en déduisez-vous ?

#### Exercice 3

Essayer d'améliorer les performances en aidant un peu plus Numba à optimiser cette fonction. Quel est le gain ?